In [12]:
import numpy as np
import pandas as pd
import pickle

In [13]:
# Chemin pour save and load les fichiers pickle
datadir = '/home/chardong/y_identification/Venv/save_pkl/Fudge_Factor/'

In [14]:
totald = pd.read_pickle(datadir+"Py8_yj_jj_mc16ade_pd122_train_w_skim_05.pkl")

In [15]:
columns = list(totald.columns)
columns

['y_Reta',
 'y_Rphi',
 'y_weta2',
 'y_fracs1',
 'y_weta1',
 'y_wtots1',
 'y_Rhad',
 'y_Rhad1',
 'y_Eratio',
 'y_deltae',
 'y_noFF_Reta',
 'y_noFF_Rphi',
 'y_noFF_weta2',
 'y_noFF_fracs1',
 'y_noFF_weta1',
 'y_noFF_wtots1',
 'y_noFF_Rhad',
 'y_noFF_Rhad1',
 'y_noFF_Eratio',
 'y_noFF_deltae',
 'y_convRadius',
 'y_convType',
 'y_pt',
 'y_eta',
 'y_phi',
 'evt_mu',
 'y_jmin_dr',
 'y_IsTight',
 'y_IsLoose',
 'y_truth_pt',
 'y_truth_eta',
 'weight',
 'truth_label']

# Fonction on shower-shape variables depending on their Fudge Factor to create the upper and bottom boundary.
Be carefull about X > X_noFF or X < X_noFF to correctly applied the function

FFplus Calculation:
    
- If X_noFF is greater than or equal to X, it means that X is lower, so we increase X by half the difference to get FFplus.
- If X_noFF is less than X, it means that X is higher, so we decrease X by half the difference to get FFplus.

FFminus Calculation:
    
- If X_noFF is greater than or equal to X, it means that X is lower, so we decrease X by half the difference to get FFminus.
- If X_noFF is less than X, it means that X is higher, so we increase X by half the difference to get FFminus.

This ensures that FFplus and FFminus reflect the upper and lower bounds of the fudge factor adjustments.

In [ ]:
def FF_combined(X, X_noFF):
    FFplus = np.where(X_noFF >= X, X + (X_noFF - X) / 2, X - (X_noFF - X) / 2)
    FFminus = np.where(X_noFF >= X, X - (X_noFF - X) / 2, X + (X_noFF - X) / 2)
    return pd.Series(FFplus), pd.Series(FFminus)

TEST

In [11]:
plusy_Reta, minusy_Reta = FF_combined(totald['y_Reta'], totald['y_noFF_Reta'])
plusy_Rphi, minusy_Rphi = FF_combined(totald['y_Rphi'], totald['y_noFF_Rphi'])
print(plusy_Reta.head(1))
print(minusy_Reta.head(1))
print(plusy_Rphi.head(1))
print(minusy_Rphi.head(1))

0    0.950085
dtype: float32
0    0.946143
dtype: float32
0    0.974662
dtype: float32
0    0.969418
dtype: float32


In [5]:
# Liste des colonnes shower shape
shower_shape_columns = ['y_Reta', 'y_Rphi', 'y_weta2', 'y_fracs1', 'y_weta1', 'y_wtots1', 'y_Rhad', 'y_Rhad1', 'y_Eratio', 'y_deltae']

# Appliquer la fonction à chaque paire de colonnes
for col in shower_shape_columns:
    noFF_col = 'y_noFF_' + col.split('_')[1]
    if noFF_col in totald.columns:
        totald[f'{col}_FFplus'], totald[f'{col}_FFminus'] = FF_combined(totald[col], totald[noFF_col])

#print(totald.head(5))

            y_Reta    y_Rphi   y_weta2  y_fracs1   y_weta1  y_wtots1  \
33407159  0.948114  0.972040  0.010898  0.157839  0.540920  1.254022   
36404334  0.972290  0.983953  0.008934  0.189961  0.595370  1.869640   
14421770  0.955383  0.972751  0.009635  0.309654  0.601149  2.073747   
4640291   0.967032  0.981241  0.009195  0.205499  0.575297  1.854882   
31454373  0.971783  0.983002  0.009087  0.171319  0.598317  1.682183   

            y_Rhad   y_Rhad1  y_Eratio    y_deltae  ...  y_wtots1_FFplus  \
33407159 -0.003790 -0.002192  0.984558   79.002472  ...              NaN   
36404334 -0.000200 -0.000055  0.995755    7.003021  ...              NaN   
14421770  0.002041  0.003310  0.983183   52.102280  ...              NaN   
4640291  -0.000106 -0.000323  0.976117   55.048897  ...              NaN   
31454373  0.001505  0.001421  0.976171  229.999527  ...              NaN   

          y_wtots1_FFminus  y_Rhad_FFplus  y_Rhad_FFminus  y_Rhad1_FFplus  \
33407159               NaN       

In [6]:
# Créer et sauvegarder le DataFrame avec FFminus
totald_FFminus = totald.copy()
for col in shower_shape_columns:
    totald_FFminus[col] = totald_FFminus[f'{col}_FFminus']
totald_FFminus.drop(columns=[f'{col}_FFplus' for col in shower_shape_columns] + [f'y_noFF_{col.split("_")[1]}' for col in shower_shape_columns], inplace=True)

with open(datadir + 'totald_FFminus.pkl', 'wb') as f:
    pickle.dump(totald_FFminus, f)
print("DataFrame avec FFminus sauvegardé.")

# Créer et sauvegarder le DataFrame avec FFplus
totald_FFplus = totald.copy()
for col in shower_shape_columns:
    totald_FFplus[col] = totald_FFplus[f'{col}_FFplus']
totald_FFplus.drop(columns=[f'{col}_FFminus' for col in shower_shape_columns] + [f'y_noFF_{col.split("_")[1]}' for col in shower_shape_columns], inplace=True)

with open(datadir + 'totald_FFplus.pkl', 'wb') as f:
    pickle.dump(totald_FFplus, f)
print("DataFrame avec FFplus sauvegardé.")

# Créer et sauvegarder le DataFrame avec noFF
totald_noFF = totald.copy()
for col in shower_shape_columns:
    noFF_col = 'y_noFF_' + col.split('_')[1]
    totald_noFF[col] = totald_noFF[noFF_col]
totald_noFF.drop(columns=[f'{col}_FFplus' for col in shower_shape_columns] + [f'{col}_FFminus' for col in shower_shape_columns], inplace=True)

with open(datadir + 'totald_noFF.pkl', 'wb') as f:
    pickle.dump(totald_noFF, f)
print("DataFrame avec noFF sauvegardé.")

DataFrame avec FFminus sauvegardé.
DataFrame avec FFplus sauvegardé.
DataFrame avec noFF sauvegardé.


In [7]:
# Liste des colonnes shower shape de base
shower_shape_columns_base = [
    'y_Reta', 'y_Rphi', 'y_weta2', 'y_fracs1', 'y_weta1',
    'y_wtots1', 'y_Rhad', 'y_Rhad1', 'y_Eratio', 'y_deltae'
]

# Supprimer les colonnes des DataFrames
totald_FFminus.drop(columns=shower_shape_columns_base, inplace=True, errors='ignore')
totald_FFplus.drop(columns=shower_shape_columns_base, inplace=True, errors='ignore')
totald_noFF.drop(columns=shower_shape_columns_base, inplace=True, errors='ignore')

# Renommer les colonnes des DataFrames
for col in shower_shape_columns_base:
    totald_FFminus.rename(columns={f'{col}_FFminus': col}, inplace=True)
    totald_FFplus.rename(columns={f'{col}_FFplus': col}, inplace=True)
    totald_noFF.rename(columns={f'y_noFF_{col.split("_")[1]}': col}, inplace=True)

# Sauvegarder les DataFrames modifiés
totald_FFminus.to_pickle(datadir + 'totald_FFminus.pkl')
totald_FFplus.to_pickle(datadir + 'totald_FFplus.pkl')
totald_noFF.to_pickle(datadir + 'totald_noFF.pkl')